<a id="top"></a>
# Restricted Zone Notifier Demo

## Check for latest version
<br><div class=danger><b>Important: Before proceeding, please run the following cell to ensure that you are running the most recent version of this sample.</b></div>

In [ ]:
# Show the current status of this and all documents with ability to update
from qarpo.catalog import DemoCatalog
import os
status = DemoCatalog(os.getcwd(), "Demo").ShowRepositoryControls()

## Introduction

This sample application demonstrates how a smart video IoT solution may be created using Intel® hardware and software tools to perform restricted zone notification.  This solution detects any number of people within a video frame and for each person determines whether they have entered into the restricted zone.  The results for each frame are stored in a text file that is later read by a second pass to annotate the input video with the inference time, the decision on whether any present people are safe and outside the restricted zone, and an additional warning message whenever a person has entered the restricted zone and is not safe.

### Key concepts
This sample application includes an example for the following:
- Application:
  - Video and image input is supported using OpenCV
  - OpenCV is used to draw bounding boxes around detected objects, labels, and other information
  - Visualization of inferred results within the output
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) and [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)


## Restricted zone notifier application
The restricted zone notifier application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on an input video to locate people within each frame and determine whether they have entered the restriction zone.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Use the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) to download the inference model IR files needed to perform inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the restricted zone notifier application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the specified input video to people.  When a person is detected, their location is compared to the predetermined restricted zone to decide whether they are "safe".  The user can specify the restricted area coordinates via command line parameters. By default the whole frame is selected. For each frame, a message containing the location, whether workers are "safe" or not, etc. is written as a line to the output text.

After the restricted zone notifier application has run on a video, a second executable is run to annotate the input video with the results from the output text file.


To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and and Intel® Arria® 10 FPGA.  After inference on the input is completed, the output is stored in the appropriate `results/<architecture>/` directory.  The results are then viewed within this Jupyter* Notebook using the `videoHTML` video playback utility.

The application and inference code for this sample is already implemented in the files: 
- First pass, detection using inference: [`restricted_zone_notifier.py`](./restricted_zone_notifier.py) and [`inference.py`](./inference.py).
- Second pass, annotation using detection results: [`ROI_writer.cpp`](./ROI_writer.cpp)

The following sections will guide you through configuring and running the restricted zone notifier demo.

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.). 
The Intel® Distribution of OpenVINO™ toolkit also includes the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) utility  to download some common inference models from the [Open Model Zoo](https://github.com/opencv/open_model_zoo). 

Run the following cell to run the Model Downloader utility with the `--print_all` argument to see all the available inference models.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The above command will also work in a terminal (with the '!' removed).</i></div>

Some of these downloaded models are already in the IR format, while others will require the Model Optimizer to be run. For our application, we will be using the [`person-detection-retail-0013`](https://github.com/opencv/open_model_zoo/tree/master/models/intel/person-detection-retail-0013) inference model, which is already in IR format. 

The format for the Model Downloader command to download a model is:
```bash
/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py \
    --name <model_name> -o <output_directory>
```
The input arguments are as follows:
- **--name** : The name of the model you want to download. It should be one of the models listed in the previous cell.
- **-o** : The output directory where to store the downloaded model. If the directory does not exist, it will be created.

Run the following cell to download the `person-detection-retail-0013` model to the `./models` directory relative to the location of this Jupyter* Notebook.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name person-detection-retail-0013 -o models
!echo "All IR files that were downloaded or created:"
!find ./models -name "*.xml" -o -name "*.bin"

As shown above from the output of the last `!find...` command, there are more than one set of IR model files (`*.xml` and `*.bin`).  The Model Downloader will download IR model files for all available precisions for the specified model, storing each in a separate directory following the format: 
```bash
<model_name>/<precision>/<<model_name.[xml|bin]>
```
We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the two most common precisions, FP32 and FP16.

### Optional exercise: View input without inference

If you are curious to see the input video, run the following cell to view the original video stream used for inference and this demo.

In [ ]:
from qarpo.demoutils import videoHTML
videoHTML('Worker Zone Detection Video', ['resources/worker-zone-detection.mp4'])

### Run the demo

Running the next cell will display an interactive user interface allowing you to submit jobs to run the demo on multiple edge compute nodes selected by hardware devices, view the output of each job, and compare performance results across jobs.

To run a job:
1. Select the desired option in the **Target node** list
2. Select the desired device in the **Target architecture** list
3. Click the **Submit** button

After the **Submit** button is clicked, a tab will appear for the new job with a label in the format "*status*: *JobID*".  Once the job status appears as "Done", the **Display output** button may be clicked to view the output for the job.

After one or more jobs are done, the performance results for each job may be plotted by clicking the **Plot results** button.  Results for each job will be potted as bar graphs for **inference time** and **frames per second**.  Lower values mean better performance for **inference time** and higher values mean better performance for **frames per second**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

Run the next cell to begin the interactive demo.

In [ ]:
# import necessary libraries
import json
import qarpo

# load job configurations for demo
with open('job_config.json') as json_file:  
    data = json.load(json_file)
    
# create and run the user job interface
job_interface = qarpo.Interface(data)
job_interface.displayUI()

## Next steps
- The complete [Restricted Zone Notifier Sample](../../developer_samples/restricted-zone-notifier-python/restricted_zone_notifier.ipynb) for this demo
- [Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) -  tutorials on using and creating Jupyter* Notebooks
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>